# [ Chapter 8 - Signals Boosting Models ]
# Signals Boosting

NOTE: This notebook depends upon the the Retrotech dataset. If you have any issues, please rerun the [Setting up the Retrotech Dataset](../ch04/1.setting-up-the-retrotech-dataset.ipynb) notebook.

In [2]:
from datetime import datetime

from pyspark.sql import SparkSession

from aips import display_product_search, get_engine
from aips.spark import create_view_from_collection
from aips.spark.dataframe import from_sql

spark = SparkSession.builder.appName("AIPS").getOrCreate()
engine = get_engine()
signals_collection = engine.get_collection("signals")

## Keyword Search with No Signals Boosting

### Figure 8.1

In [1]:
%run chapters/ch04/1.setting-up-the-retrotech-dataset.ipynb

<class 'engines.opensearch.OpenSearchEngine.OpenSearchEngine'>
Cloning into 'retrotech'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 19 (delta 0), reused 17 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 48.28 MiB | 12.05 MiB/s, done.
Already up to date.
products.csv
signals.csv
"upc","name","manufacturer","short_description","long_description"
"096009010836","Fists of Bruce Lee - Dolby - DVD", , , 
"043396061965","The Professional - Widescreen Uncut - DVD", , , 
"085391862024","Pokemon the Movie: 2000 - DVD", , , 
"067003016025","Summerbreeze - CD","Nettwerk", , 
"731454813822","Back for the First Time [PA] - CD","Def Jam South", , 
"024543008200","Big Momma's House - Widescreen - DVD", , , 
"031398751823","Kids - DVD", , , 
"037628413929","20 Grandes Exitos - CD","Sony Discos Inc.", , 
"060768972223","Power Of Trinity (Box) - CD","Sanctuary Records", , 
Wiping

"query_id","user","type","target","signal_time"
"u2_0_1","u2","query","nook","2019-07-31 08:49:07.3116"
"u2_1_2","u2","query","rca","2020-05-04 08:28:21.1848"
"u3_0_1","u3","query","macbook","2019-12-22 00:07:07.0152"
"u4_0_1","u4","query","Tv antenna","2019-08-22 23:45:54.1030"
"u5_0_1","u5","query","AC power cord","2019-10-20 08:27:00.1600"
"u6_0_1","u6","query","Watch The Throne","2019-09-18 11:59:53.7470"
"u7_0_1","u7","query","Camcorder","2020-02-25 13:02:29.3089"
"u9_0_1","u9","query","wireless headphones","2020-04-26 04:26:09.7198"
"u10_0_1","u10","query","Xbox","2019-09-13 16:26:12.0132"
Wiping "signals" collection
Creating "signals" collection
Loading data/retrotech/signals.csv
Schema: 
root
 |-- query_id: string (nullable = true)
 |-- user: string (nullable = true)
 |-- type: string (nullable = true)
 |-- target: string (nullable = true)
 |-- signal_time: timestamp (nullable = true)

Successfully written 2172605 documents


In [3]:
# %load -s product_search_request aips/search_requests
def product_search_request(query, param_overrides={}):
    request = {"query": query,
               "query_fields": ["name", "manufacturer", "long_description"],
               "return_fields": ["upc", "name", "manufacturer",
                                 "short_description", "score"],
               "limit": 5,
               "order_by": [("score", "desc"), ("upc", "asc")]}
    return request | param_overrides

In [4]:
products_collection = engine.get_collection("products")
query = "ipad"
request = product_search_request(query)
response = products_collection.search(**request)
display_product_search(query, response["docs"])

## Create Signals Boosts (Signals Aggregation)

### Basic Signals Boosting Model

In [5]:
def aggregate_signals(signals_collection, collection_name, signals_agg_query):
    print("Aggregating Signals to Create Signals Boosts...")
    create_view_from_collection(signals_collection, "signals")
    collection = engine.create_collection(collection_name)
    collection.write(from_sql(signals_agg_query))
    collection.commit()
    print("Signals Aggregation Completed!")
    return collection

## Search with Signals Boosts Applied

### Signals Boosting Query

## Raw Signals Boosts (Case-sensitive)

### Listing 8.1

In [6]:
# %load -s search_for_boosts aips/search_requests
def search_for_boosts(query, collection, query_field="query"):
    boosts_request = {"query": query,
                      "query_fields": [query_field],
                      "return_fields": ["query", "doc", "boost"],
                      "limit": 20,
                      "order_by": [("boost", "desc")]}
    response = collection.search(**boosts_request)
    return response["docs"]

In [7]:
def create_boosting_collection(collection_name):
    basic_signals_aggregation_query = """
    SELECT q.target AS query, c.target AS doc,
    COUNT(c.target) AS boost
    FROM signals c LEFT JOIN signals q
    ON c.query_id = q.query_id
    WHERE c.type = 'click' AND q.type = 'query'
    GROUP BY query, doc
    ORDER BY boost DESC
    """
    collection = engine.get_collection(collection_name)
    return aggregate_signals(collection, "basic_signals_boosts", basic_signals_aggregation_query)

In [8]:
def show_raw_boosted_queries(boost_documents):
    boosted_queries = "\n".join([f'"{doc["query"]}" : {doc["boost"]}'
                                 for doc in boost_documents])
    print("Raw Boosted Queries")
    print(boosted_queries)

In [9]:
signals_boosting_collection = create_boosting_collection("signals")

Aggregating Signals to Create Signals Boosts...
Wiping "basic_signals_boosts" collection
Creating "basic_signals_boosts" collection
Successfully written 197168 documents
Signals Aggregation Completed!


In [10]:
query = "885909457588" #most popular iPad model
signals_docs = search_for_boosts(query, signals_boosting_collection, "doc")
show_raw_boosted_queries(signals_docs)    

Raw Boosted Queries
"iPad" : 1050
"ipad" : 966
"Ipad" : 829
"iPad 2" : 509
"ipad 2" : 347
"Ipad2" : 261
"ipad2" : 238
"Ipad 2" : 213
"I pad" : 203
"i pad" : 133
"IPad" : 77
"Apple" : 76
"I pad 2" : 60
"apple ipad" : 55
"Apple iPad" : 53
"ipads" : 43
"tablets" : 42
"apple" : 41
"iPads" : 38
"i pad 2" : 38


## Normalized Signals Boosting (Case-insensitive)

### Listing 8.2

In [11]:
normalized_signals_aggregation_query = """
SELECT LOWER(q.target) AS query,
c.target AS doc, COUNT(c.target) AS boost
FROM signals c LEFT JOIN signals q ON c.query_id = q.query_id
WHERE c.type = 'click' AND q.type = 'query'
GROUP BY query, doc
ORDER BY boost DESC"""

normalized_collection = \
  aggregate_signals(signals_collection, "normalized_signals_boosts",
                    normalized_signals_aggregation_query)

Aggregating Signals to Create Signals Boosts...
Wiping "normalized_signals_boosts" collection
Creating "normalized_signals_boosts" collection
Successfully written 152251 documents
Signals Aggregation Completed!


In [12]:
query = "885909457588" #most popular iPad model
signals_docs = search_for_boosts(query, normalized_collection, "doc")
show_raw_boosted_queries(signals_docs)

Raw Boosted Queries
"ipad" : 2939
"ipad 2" : 1104
"ipad2" : 540
"i pad" : 341
"apple ipad" : 152
"ipads" : 123
"apple" : 118
"i pad 2" : 99
"tablets" : 67
"tablet" : 61
"ipad 1" : 52
"apple ipad 2" : 27
"hp touchpad" : 26
"ipaq" : 20
"i pad2" : 19
"wi" : 19
"apple computers" : 18
"apple i pad" : 15
"ipad 2 16gb" : 15
"samsung galaxy" : 14


## Star Wars Search before User Manipulation

### Figure 8.2

In [13]:
# %load -s create_boosts_query,boosted_product_search_request aips/search_requests
def create_boosts_query(boost_documents):
    print("Boost Documents:")
    display(boost_documents)
    boosts = " ".join([f'"{b["doc"]}"^{b["boost"]}' 
                       for b in boost_documents])
    print(f"\nBoost Query: \n{boosts}\n")
    return boosts

In [14]:
def boosted_product_search_request(query, collection, boost_field=None):
    signals_documents = search_for_boosts(query, collection)
    signals_boosts = create_boosts_query(signals_documents)
    boosted_request = product_search_request(query)
    if boost_field:
        signals_boosts = (boost_field, signals_boosts)
    boosted_request["query_boosts"] = signals_boosts
    return boosted_request

In [15]:
query = '"star wars"'
boosted_request = boosted_product_search_request(query, normalized_collection, "upc")
boosted_request["filter"] = [("-upc", "45626176")]

response = products_collection.search(**boosted_request)
display_product_search("star wars", response["docs"])

Boost Documents:


[{'query': 'star wars',
  'doc': '024543742180',
  'boost': 1490,
  'id': 'NwUhd5IBPi5UfNaKgNyx',
  'score': None},
 {'query': 'star wars blu ray',
  'doc': '024543742180',
  'boost': 338,
  'id': 'cgUhd5IBPi5UfNaKgNyx',
  'score': None},
 {'query': 'star wars',
  'doc': '400032015667',
  'boost': 186,
  'id': '5AUhd5IBPi5UfNaKgNyx',
  'score': None},
 {'query': 'star wars',
  'doc': '024543742074',
  'boost': 127,
  'id': 'ZQUhd5IBPi5UfNaKgN2x',
  'score': None},
 {'query': 'star wars',
  'doc': '024543559856',
  'boost': 117,
  'id': 'lgUhd5IBPi5UfNaKgN2x',
  'score': None},
 {'query': 'star wars',
  'doc': '014633169546',
  'boost': 107,
  'id': 'zwUhd5IBPi5UfNaKgN2x',
  'score': None},
 {'query': 'star wars bluray',
  'doc': '024543742180',
  'boost': 87,
  'id': 'aAUhd5IBPi5UfNaKgN6x',
  'score': None},
 {'query': 'star wars blue ray',
  'doc': '024543742180',
  'boost': 83,
  'id': 'jwUhd5IBPi5UfNaKgN6x',
  'score': None},
 {'query': 'star wars',
  'doc': '024543742098',
  'boost


Boost Query: 
"024543742180"^1490 "024543742180"^338 "400032015667"^186 "024543742074"^127 "024543559856"^117 "014633169546"^107 "024543742180"^87 "024543742180"^83 "024543742098"^81 "024543781875"^67 "400032015667"^63 "024543742074"^49 "024543742180"^44 "024543742180"^42 "024543742180"^39 "023272342654"^39 "024543742180"^36 "014633169522"^32 "024543560067"^30 "885370332889"^28



## Simulating a malicious user

### Listing 8.3

In [16]:
signals_collection = engine.get_collection("signals")
spam_user = "u8675309"
spam_query = "star wars"
spam_signal_boost_doc_upc = "45626176" #doc for a "trash compactor" from someone who wants star wars is rubbish!

signal_docs = []
for num in range(5000): #generate 5,000 query and click signals
    query_id = f"u8675309_0_{num}"
    query_signal = { 
        "query_id": query_id,
        "user": spam_user,
        "type": "query",
        "target": spam_query,
        "signal_time": datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
        "id": f"spam_signal_query_{num}"
    }
    click_signal = { 
        "query_id": query_id,
        "user": spam_user,
        "type": "click",
        "target": spam_signal_boost_doc_upc,
        "signal_time": datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
        "id": f"spam_signal_click_{num}"
    }
    signal_docs.extend([click_signal, query_signal])

signals_collection.add_documents(signal_docs)
#re-run the basic signals aggregation to process the malicious clicks

spam_signals_collection = \
    aggregate_signals(signals_collection, "signals_boosts_with_spam",
                      normalized_signals_aggregation_query)

Py4JJavaError: An error occurred while calling o110.save.
: org.opensearch.hadoop.rest.OpenSearchHadoopNoNodesLeftException: Connection error (check network and/or proxy settings)- all nodes failed; tried [[http://172.20.0.4:9200]] 
	at org.opensearch.hadoop.rest.NetworkClient.execute(NetworkClient.java:170)
	at org.opensearch.hadoop.rest.RestClient.execute(RestClient.java:443)
	at org.opensearch.hadoop.rest.RestClient.execute(RestClient.java:439)
	at org.opensearch.hadoop.rest.RestClient.executeNotFoundAllowed(RestClient.java:451)
	at org.opensearch.hadoop.rest.RestClient.deleteByQuery(RestClient.java:518)
	at org.opensearch.hadoop.rest.RestRepository.delete(RestRepository.java:383)
	at org.opensearch.spark.sql.OpenSearchRelation.insert(DefaultSource.scala:559)
	at org.opensearch.spark.sql.DefaultSource.createRelation(DefaultSource.scala:107)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:47)
	at org.apache.spark.sql.execut

## Impact of Spam on Search Results

### Listing 8.4

In [ ]:
# function boosted_product_search_request defined above before figure 8.2 code

query = '"star wars"'
boosted_request = boosted_product_search_request(query,
                      spam_signals_collection, "upc")
response = products_collection.search(**boosted_request)
display_product_search(query, response["docs"])

Boost Documents:


[{'query': 'star wars', 'doc': '45626176', 'boost': 5000},
 {'query': 'star wars', 'doc': '024543742180', 'boost': 1490},
 {'query': 'star wars', 'doc': '400032015667', 'boost': 186},
 {'query': 'star wars', 'doc': '024543742074', 'boost': 127},
 {'query': 'star wars', 'doc': '024543559856', 'boost': 117},
 {'query': 'star wars', 'doc': '014633169546', 'boost': 107},
 {'query': 'star wars', 'doc': '024543742098', 'boost': 81},
 {'query': 'star wars', 'doc': '024543781875', 'boost': 67},
 {'query': 'star wars', 'doc': '023272342654', 'boost': 39},
 {'query': 'star wars', 'doc': '014633169522', 'boost': 32},
 {'query': 'star wars', 'doc': '024543560067', 'boost': 30},
 {'query': 'star wars', 'doc': '883929154012', 'boost': 24},
 {'query': 'star wars', 'doc': '023272342630', 'boost': 20},
 {'query': 'star wars', 'doc': '708431390614', 'boost': 18},
 {'query': 'star wars', 'doc': '024543263739', 'boost': 18},
 {'query': 'star wars', 'doc': '886973561621', 'boost': 17},
 {'query': 'star war


Boost Query: 
"45626176"^5000 "024543742180"^1490 "400032015667"^186 "024543742074"^127 "024543559856"^117 "014633169546"^107 "024543742098"^81 "024543781875"^67 "023272342654"^39 "014633169522"^32 "024543560067"^30 "883929154012"^24 "023272342630"^20 "708431390614"^18 "024543263739"^18 "886973561621"^17 "023272341381"^16 "030206742121"^14 "885370332889"^12 "883929106172"^8



## Fighting Signal Spam through User-level Deduplication

### Listing 8.5

In [ ]:
#One Signal per User - Anti-Spam
anti_spam_aggregation_query = """
SELECT query, doc, COUNT(doc) AS boost FROM (
  SELECT c.user unique_user, LOWER(q.target) AS query, c.target AS doc,
  MAX(c.signal_time) AS boost
  FROM signals c LEFT JOIN signals q ON c.query_id = q.query_id
  WHERE c.type = 'click' AND q.type = 'query'
  GROUP BY unique_user, query, doc)
GROUP BY query, doc
ORDER BY boost DESC"""

anti_spam_collection = \
    aggregate_signals(signals_collection, "signals_boosts_anti_spam",
                      anti_spam_aggregation_query)

Aggregating Signals to Create Signals Boosts...
Wiping "signals_boosts_anti_spam" collection
Creating "signals_boosts_anti_spam" collection
Status: Success
Successfully written 152252 documents
Signals Aggregation Completed!


In [ ]:
query = '"star wars"'

boosted_request = boosted_product_search_request(query, anti_spam_collection)
response = products_collection.search(**boosted_request)

print(products_collection.transform_request(**boosted_request))
display_product_search(query, response["docs"])

Boost Documents:
[{'query': 'star wars', 'doc': '024543742180', 'boost': 1489}, {'query': 'star wars', 'doc': '400032015667', 'boost': 186}, {'query': 'star wars', 'doc': '024543742074', 'boost': 127}, {'query': 'star wars', 'doc': '024543559856', 'boost': 117}, {'query': 'star wars', 'doc': '014633169546', 'boost': 107}, {'query': 'star wars', 'doc': '024543742098', 'boost': 81}, {'query': 'star wars', 'doc': '024543781875', 'boost': 67}, {'query': 'star wars', 'doc': '023272342654', 'boost': 39}, {'query': 'star wars', 'doc': '014633169522', 'boost': 32}, {'query': 'star wars', 'doc': '024543560067', 'boost': 30}, {'query': 'star wars', 'doc': '883929154012', 'boost': 24}, {'query': 'star wars', 'doc': '023272342630', 'boost': 20}, {'query': 'star wars', 'doc': '708431390614', 'boost': 18}, {'query': 'star wars', 'doc': '024543263739', 'boost': 18}, {'query': 'star wars', 'doc': '886973561621', 'boost': 17}, {'query': 'star wars', 'doc': '023272341381', 'boost': 16}, {'query': 'star 

## Mixing multiple signal types

### Listing 8.6

In [ ]:
#One Signal per User - Anti-Spam
#Sometimes needs rerunning
mixed_signal_types_aggregation_query = """
SELECT query, doc, ((1 * click_boost) + (10 * add_to_cart_boost) +
                    (25 * purchase_boost)) AS boost FROM (
  SELECT query, doc, 
    SUM(click) AS click_boost,
    SUM(add_to_cart) AS add_to_cart_boost,
    SUM(purchase) AS purchase_boost FROM (  
      SELECT lower(q.target) AS query, cap.target AS doc, 
        IF(cap.type = 'click', 1, 0) AS click, 
        IF(cap.type = 'add-to-cart', 1, 0) AS  add_to_cart, 
        IF(cap.type = 'purchase', 1, 0) AS purchase
      FROM signals cap LEFT JOIN signals q on cap.query_id = q.query_id
      WHERE (cap.type != 'query' AND q.type = 'query')
    ) AS raw_signals
  GROUP BY query, doc) AS per_type_boosts"""

type_weighted_collection = \
  aggregate_signals(signals_collection, "signals_boosts_weighted_types",
                    mixed_signal_types_aggregation_query)

Aggregating Signals to Create Signals Boosts...
Wiping "signals_boosts_weighted_types" collection
Creating "signals_boosts_weighted_types" collection
Status: Success
Successfully written 152252 documents
Signals Aggregation Completed!


## Time Decay

### Listing 8.7

In [ ]:
half_life_days = 30
target_date = "2020-06-01" #date of latest signal in our dataset. In live system use now().
signal_weight = 1 #can make this a function to differentiate weights for different signal types

time_decay_aggregation = f"""
SELECT query, doc, sum(time_weighted_boost) AS boost FROM (
  SELECT user, query, doc, {signal_weight} *
  POW(0.5, (DATEDIFF('{target_date}', signal_time) / 
            {half_life_days}))
  AS time_weighted_boost FROM (
    SELECT c.user AS user, lower(q.target) AS query, c.target AS doc,
    MAX(c.signal_time) as signal_time
    FROM signals c LEFT JOIN signals q ON c.query_id = q.query_id
    WHERE c.type = 'click' AND q.type = 'query'
          AND c.signal_time <= '{target_date}'
    GROUP BY c.user, q.target, c.target
  ) AS raw_signals 
) AS time_weighted_signals
GROUP BY query, doc
ORDER BY boost DESC"""

time_weighted_collection = \
    aggregate_signals(signals_collection, "signals_boosts_time_weighted", 
                      time_decay_aggregation)

Aggregating Signals to Create Signals Boosts...
Wiping "signals_boosts_time_weighted" collection
Creating "signals_boosts_time_weighted" collection
Status: Success
Successfully written 152251 documents
Signals Aggregation Completed!


In [ ]:
query = '"star wars"'

boosted_request = boosted_product_search_request(query, time_weighted_collection)
print(products_collection.transform_request(**boosted_request))

response = products_collection.search(**boosted_request)
display_product_search(query, response["docs"])

Boost Documents:


[{'query': 'star wars', 'doc': '024543742180', 'boost': 50.275649588684246},
 {'query': 'star wars', 'doc': '023272342654', 'boost': 14.386081175228192},
 {'query': 'star wars', 'doc': '024543742074', 'boost': 10.978708113139781},
 {'query': 'star wars', 'doc': '883929154012', 'boost': 10.22196625832087},
 {'query': 'star wars', 'doc': '023272342630', 'boost': 8.08610476717647},
 {'query': 'star wars', 'doc': '886973561621', 'boost': 7.196654524422255},
 {'query': 'star wars', 'doc': '024543742098', 'boost': 6.411867877358415},
 {'query': 'star wars', 'doc': '030206742121', 'boost': 6.0339109822265105},
 {'query': 'star wars', 'doc': '014633169546', 'boost': 5.7322561332539275},
 {'query': 'star wars', 'doc': '708431390614', 'boost': 5.200090923395828},
 {'query': 'star wars', 'doc': '024543559856', 'boost': 4.79595089562249},
 {'query': 'star wars', 'doc': '023272341381', 'boost': 2.919945959727525},
 {'query': 'star wars', 'doc': '014633169522', 'boost': 2.2727988663619274},
 {'query


Boost Query: 
"024543742180"^50.275649588684246 "023272342654"^14.386081175228192 "024543742074"^10.978708113139781 "883929154012"^10.22196625832087 "023272342630"^8.08610476717647 "886973561621"^7.196654524422255 "024543742098"^6.411867877358415 "030206742121"^6.0339109822265105 "014633169546"^5.7322561332539275 "708431390614"^5.200090923395828 "024543559856"^4.79595089562249 "023272341381"^2.919945959727525 "014633169522"^2.2727988663619274 "886971404722"^2.244924096618746 "024543560067"^1.6538835179903377 "400032015667"^1.4480483379761804 "883929153992"^0.9339199409434634 "023272341633"^0.8908987181403393 "024543781875"^0.7423932331629598 "024543212768"^0.7405487761432821

{'query': '"star wars"', 'limit': 5, 'params': {'qf': ['name', 'manufacturer', 'long_description'], 'boost': 'sum(1,query($boost_query))', 'boost_query': '"024543742180"^50.275649588684246 "023272342654"^14.386081175228192 "024543742074"^10.978708113139781 "883929154012"^10.22196625832087 "023272342630"^8.0861047

## Index-time Signals Boosting

In [ ]:
from aips.data_loaders.products import load_dataframe

#Create collection with index-time signals boosts field
boosted_products_collection = engine.create_collection("products_with_signals_boosts")
boosted_products_collection.write(load_dataframe("data/retrotech/products.csv"))

Wiping "products_with_signals_boosts" collection
Creating "products_with_signals_boosts" collection
Adding "boosts" field type to collection
Status: Success
Loading Products
Schema: 
root
 |-- upc: long (nullable = true)
 |-- name: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- long_description: string (nullable = true)

Successfully written 48194 documents


### Listing 8.8

In [ ]:
normalized_collection = engine.get_collection("normalized_signals_boosts")
create_view_from_collection(normalized_collection,
                       normalized_collection.name)

boosted_products_collection = \
    engine.get_collection("products_with_signals_boosts")
create_view_from_collection(boosted_products_collection,
                       boosted_products_collection.name)

#insert all keywords with signals boosts for this document into a new "signals_boosts" field
boosts_query = f"""
SELECT p.*, b.signals_boosts FROM (
    SELECT doc, CONCAT_WS(',', COLLECT_LIST(CONCAT(query, '|', boost)))
    AS signals_boosts FROM {normalized_collection.name} GROUP BY doc
) b INNER JOIN {boosted_products_collection.name} p ON p.upc = b.doc"""

#save the products back to the products collection, with the updated signals boosts added
boosted_products_collection.write(from_sql(boosts_query))

Successfully written 15559 documents


## Index-time Signals Boosting Query

### Listing 8.9

In [ ]:
def get_boosted_search_request(query, boost_field):
    request = product_search_request(query)
    request["index_time_boost"] = (boost_field, query)
    return request

In [ ]:
query = "ipad"
boosted_query = get_boosted_search_request(query, "signals_boosts")
response = boosted_products_collection.search(**boosted_query)
display_product_search(query, response["docs"])

Up next: Chapter 9 - [Personalized Search](../ch09/1.personalization.ipynb)